In [1]:
import time
from bs4 import BeautifulSoup as bs
import pandas as pd
import re
import os

from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
# 리뷰 끝까지 무한 스크롤
def scroll(modal, rep_num):
    try:        
        # 스크롤 높이 받아오기
        last_height = driver.execute_script("return arguments[0].scrollHeight", modal)
        for i in range(rep_num):
            pause_time = 3
            # 최하단까지 스크롤
            driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight);", modal)
            # 페이지 로딩 대기
            time.sleep(pause_time)
            # 무한 스크롤 동작을 위해 살짝 위로 스크롤
            driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight-50);", modal)
            time.sleep(pause_time)
            # 스크롤 높이 새롭게 받아오기
            new_height = driver.execute_script("return arguments[0].scrollHeight", modal)
            
            last_height = new_height
    except Exception as e:
        print("에러 발생: ", e)

In [3]:
# 날짜 크롤링
def date_func(date_list, start_num, end_num):
    for i in range(start_num, end_num):
        date_x_path = '//*[@id="yDmH0d"]/div[4]/div[2]/div/div/div/div/div[2]/div/div[1]/div[{}]/header/div[2]/span'.format(str(i))
        try:
            date = driver.find_element(By.XPATH, date_x_path)
            if date:
                date_text = date.get_attribute('innerText')
                date_list.append(date_text)             
        except:
            date_list.append('-') 
       
    return date_list

In [4]:
# 이름 크롤링
def name_func(name_list, start_num, end_num):
    for i in range(start_num, end_num):
        name_x_path = '//*[@id="yDmH0d"]/div[4]/div[2]/div/div/div/div/div[2]/div/div[1]/div[{}]/header/div[1]/div[1]/div'.format(str(i))
        try:
            name = driver.find_element(By.XPATH, name_x_path)
            if name:
                name_text = name.get_attribute('innerText')
                name_list.append(name_text)
        except:
            name_list.append('-')
    
    return name_list

In [5]:
#별점 크롤링
def star_func(star_list, start_num, end_num):
    for i in range(start_num, end_num):
        star_x_path = '//*[@id="yDmH0d"]/div[4]/div[2]/div/div/div/div/div[2]/div/div[1]/div[{}]/header/div[2]/div'.format(str(i))
        try:
            star = driver.find_element(By.XPATH, star_x_path)
            if star:
                star_text = star.get_attribute('aria-label')
                star_list.append(star_text[10])
        except:
            star_list.append('-')
        
    return star_list

In [6]:
# 좋아요 크롤링
def like_func(like_list, start_num, end_num):
    for i in range(start_num, end_num):
        like_x_path = '//*[@id="yDmH0d"]/div[4]/div[2]/div/div/div/div/div[2]/div/div[1]/div[{}]/div[2]/div'.format(str(i))
        try:
            like = driver.find_element(By.XPATH, like_x_path)
            like_text = like.get_attribute('innerText')
            like_num = re.findall(r'\d+명', str(like_text)) # 숫자+명 형태로 나옴
            like_num = int(like_num[0][:-1])
            like_list.append(like_num)
        except:
            like_list.append('-')
            
    return like_list

In [7]:
#댓글 크롤링
def review_func(review_list, start_num, end_num):
    for i in range(start_num, end_num):
        review_x_path = '//*[@id="yDmH0d"]/div[4]/div[2]/div/div/div/div/div[2]/div/div[1]/div[{}]/div[1]'.format(str(i))
        try:
            review = driver.find_element(By.XPATH, review_x_path)
            if review:
                review_text = review.get_attribute('innerText')
                review_list.append(review_text)
        except:
            review_list.append('-')

    return review_list

## 크롤링 시작

In [8]:
# 드라이버 오픈
driver = webdriver.Chrome("C:/Users/Residence5/Downloads/pbl/chromedriver.exe")
url = 'https://play.google.com/store/apps/details?id=com.tinder'

driver.get(url)
driver.maximize_window()
time.sleep(5)

# 리뷰 창 클릭
review_window = driver.find_element(By.CSS_SELECTOR, 'div.qZmL0 > c-wiz > section > header > div > div > button > div.VfPpkd-Bz112c-J1Ukfc-LhBDec')
driver.execute_script("arguments[0].click();", review_window)

In [ ]:
# 스크롤 다운할 modal 창
modal = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.XPATH, "//div[@class='fysCi']")))

# down번 내린 후 데이터 수집
down = 1

# 최종 데이터프레임
f_df = pd.DataFrame(columns=['date', 'name', 'score', 'like', 'review'])

# 데이터 수집
for n in range(1, 10000):
    driver.delete_all_cookies() # 크롬의 out of memory 방지하기 위해 쿠키 삭제
    time.sleep(2)
    
    start = time.time() # 소요시간 측정
    
    # 스크롤
    scroll(modal, down) # 0:40,1:40+80*1, 2: 40+80*2, 3:40+80*3, ... 30:40+80*30 .. 한번 내릴 때마다 +80
    
    # 데이터 본격 수집
    start_num=40+80*down*(n-1)+1 # 시작 인덱스
    end_num=40+80*down*n+1 # 마지막 인덱스+1
    
    if n==1:
        start_num = 1
    
    print(start_num, end_num-1)
    
    print('날짜')
    d_list = date_func(date_list=[], start_num=start_num, end_num=end_num) # 날짜
    print('이름')
    n_list = name_func(name_list=[], start_num=start_num, end_num=end_num) # 이름
    print('별점')
    s_list = star_func(star_list=[], start_num=start_num, end_num=end_num) # 별점
    print('좋아요')
    l_list = like_func(like_list=[], start_num=start_num, end_num=end_num) # 좋아요
    print('댓글')
    r_list = review_func(review_list=[], start_num=start_num, end_num=end_num) # 댓글
   
    # 데이터 프레임 형성
    data={'date': d_list, 'name': n_list, 'score': s_list, 'like': l_list, 'review': r_list}
    df = pd.DataFrame(data)
    f_df = pd.concat([f_df, df], ignore_index=True)
    
    # 파일 저장
    data_dir = './data/tinder_google_review_{}.csv'.format(str(end_num-1))
    f_df.to_csv(data_dir, encoding='utf-16', index=False)
    
    print('소요시간: ', round((time.time()-start)/60, 2), '분')
    print('=================================')

1 120
날짜
이름
별점
좋아요
댓글
소요시간:  0.32 분
121 200
날짜
이름
별점
좋아요
댓글
소요시간:  0.28 분
201 280
날짜
이름
별점
좋아요
댓글
소요시간:  0.31 분
281 360
날짜
이름
별점
좋아요
댓글
소요시간:  0.33 분
361 440
날짜
이름
별점
좋아요
댓글
소요시간:  0.35 분
441 520
날짜
이름
별점
좋아요
댓글
소요시간:  0.37 분
521 600
날짜
이름
별점
좋아요
댓글
소요시간:  0.4 분
601 680
날짜
이름
별점
좋아요
댓글
소요시간:  0.41 분
681 760
날짜
이름
별점
좋아요
댓글
소요시간:  0.43 분
761 840
날짜
이름
별점
좋아요
댓글
소요시간:  0.45 분
841 920
날짜
이름
별점
좋아요
댓글
소요시간:  0.46 분
921 1000
날짜
이름
별점
좋아요
댓글
소요시간:  0.48 분
1001 1080
날짜
이름
별점
좋아요
댓글
소요시간:  0.5 분
1081 1160
날짜
이름
별점
좋아요
댓글
소요시간:  0.52 분
1161 1240
날짜
이름
별점
좋아요
댓글
소요시간:  0.55 분
1241 1320
날짜
이름
별점
좋아요
댓글
소요시간:  0.57 분
1321 1400
날짜
이름
별점
좋아요
댓글
소요시간:  0.58 분
1401 1480
날짜
이름
별점
좋아요
댓글
소요시간:  0.6 분
1481 1560
날짜
이름
별점
좋아요
댓글
소요시간:  0.62 분
1561 1640
날짜
이름
별점
좋아요
댓글
소요시간:  0.64 분
1641 1720
날짜
이름
별점
좋아요
댓글
소요시간:  0.71 분
1721 1800
날짜
이름
별점
좋아요
댓글
소요시간:  0.7 분
1801 1880
날짜
이름
별점
좋아요
댓글
소요시간:  0.77 분
1881 1960
날짜
이름
별점
좋아요
댓글
소요시간:  0.8 분
1961 2040
날짜
이름
별점
좋아요
댓글
소요시간:  0.76 분
2041 2120
날짜
이름
별점
좋아요
댓글
소요시간

In [ ]:
# 참고. 반복횟수 알기
down=10
n=1
while True:
    a = 40+80*down*n
    
    if a < 4740000:
        n+=1
    else:
        print(a, n)
        break    

In [ ]:
len(d_list), len(n_list), len(s_list), len(l_list), len(r_list)

### 정확한 리뷰 수를 몰라 for문이 중간에 끊긴 경우

In [ ]:
driver.delete_all_cookies() # 크롬의 out of memory 방지하기 위해 쿠키 삭제
time.sleep(1)

start = time.time() # 소요시간 측정

# 데이터 본격 수집
start_num= # 시작 인덱스
end_num= # 마지막 인덱스+1

print(start_num, end_num)

print('날짜')
d_list = date_func(date_list=[], start_num=start_num, end_num=end_num) # 날짜
print('이름')
n_list = name_func(name_list=[], start_num=start_num, end_num=end_num) # 이름
print('별점')
s_list = star_func(star_list=[], start_num=start_num, end_num=end_num) # 별점
print('좋아요')
l_list = like_func(like_list=[], start_num=start_num, end_num=end_num) # 좋아요
print('댓글')
r_list = review_func(review_list=[], start_num=start_num, end_num=end_num) # 댓글

# 데이터 프레임 형성
data={'date': d_list, 'name': n_list, 'score': s_list, 'like': l_list, 'review': r_list}
df = pd.DataFrame(data)
f_df = pd.concat([f_df, df], ignore_index=True)

# 파일 저장
data_dir = './data/tinder_google_review_{}.csv'.format(str(end_num-1))
f_df.to_csv(data_dir, encoding='utf-16', index=False)

print('소요시간: ', round((time.time()-start)/60, 2), '분')
print('=================================')